## 📥 Step 1: Clone Repository and Setup

In [ ]:
# Clone the repository (update with your GitHub URL)
!git clone https://github.com/faisalnoufal/bgbye.git
%cd bgbye/server

print("\n✅ Repository cloned successfully!")

## 📦 Step 2: Install Dependencies

This will install all required packages (~2-3 minutes)

In [ ]:
# Install core dependencies
!pip install -q fastapi uvicorn python-multipart
!pip install -q pillow numpy requests

# Install PyTorch (CPU version for Colab)
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cpu

# Install ML models and utilities
!pip install -q carvekit transformers huggingface-hub

# Install ngrok for public URL
!pip install -q pyngrok

# Install nest-asyncio for Colab compatibility
!pip install -q nest-asyncio

print("\n✅ All dependencies installed successfully!")

## 🔐 Step 3: Authenticate with HuggingFace

RMBG-2.0 requires HuggingFace authentication.

**Before running this cell:**
1. Go to https://huggingface.co/settings/tokens
2. Create a token (if you don't have one)
3. Request access to https://huggingface.co/briaai/RMBG-2.0
4. Run the cell below and paste your token

In [ ]:
from huggingface_hub import login

# This will prompt for your HuggingFace token
login()

print("\n✅ HuggingFace authentication successful!")

## 🌐 Step 4: Setup Ngrok Tunnel

This creates a public URL for your API.

**Optional:** For better limits, get a free ngrok token from https://dashboard.ngrok.com/get-started/your-authtoken

In [ ]:
from pyngrok import ngrok

# Optional: Add your ngrok authtoken for better limits (no session timeout)
# Uncomment and add your token:
# ngrok.set_auth_token("YOUR_NGROK_TOKEN")

# Kill any existing tunnels
ngrok.kill()

# Create tunnel on port 9876
public_url = ngrok.connect(9876, bind_tls=True)

print("\n" + "="*60)
print("🌐 PUBLIC API URL:")
print(f"   {public_url}")
print("\n📝 API ENDPOINT:")
print(f"   {public_url}/remove_background/")
print("\n📖 API DOCUMENTATION:")
print(f"   {public_url}/docs")
print("\n📊 Ngrok Dashboard:")
print("   http://127.0.0.1:4040")
print("="*60)
print("\n✅ Copy the public URL above to use in your frontend!")

## 🚀 Step 5: Start the FastAPI Server

This will:
1. Start the FastAPI server
2. Load all 3 models (first time may take 2-3 minutes)
3. Make the API publicly accessible

**Keep this cell running!** Stop it only when you're done.

In [ ]:
import nest_asyncio
import uvicorn
from threading import Thread
import time

# Allow nested event loops (required for Colab)
nest_asyncio.apply()

# Function to run the server
def run_server():
    uvicorn.run("server:app", host="0.0.0.0", port=9876, log_level="info")

# Start server in background thread
server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

print("\n⏳ Starting server and loading models...")
print("   (First-time model downloads may take 2-3 minutes)\n")
time.sleep(10)

print("\n" + "="*60)
print("✅ SERVER IS RUNNING!")
print("="*60)
print(f"\n🔗 Local URL:  http://localhost:9876")
print(f"🔗 Public URL: {public_url}")
print(f"\n📝 API Endpoint: {public_url}/remove_background/")
print(f"📖 API Docs:     {public_url}/docs")
print("\n⚠️  IMPORTANT: Keep this cell running to keep the server alive!")
print("="*60)

# Keep alive loop
try:
    while True:
        time.sleep(60)
        print("⏰ Server is running...", end='\r')
except KeyboardInterrupt:
    print("\n⛔ Server stopped.")

## 🧪 Step 6: Test the API (Optional)

Upload an image and test background removal with RMBG-2.0

In [ ]:
import requests
from google.colab import files
from PIL import Image
from IPython.display import display
import io

# Upload a test image
print("📤 Upload an image to test:")
uploaded = files.upload()

if uploaded:
    # Get the first uploaded file
    filename = list(uploaded.keys())[0]
    image_bytes = uploaded[filename]
    
    # Display original
    print("\n📷 Original Image:")
    display(Image.open(io.BytesIO(image_bytes)))
    
    # Process with API
    print("\n⏳ Processing with RMBG-2.0...")
    response = requests.post(
        f"{public_url}/remove_background/",
        files={"file": (filename, image_bytes)},
        data={"method": "rmbg"}
    )
    
    if response.status_code == 200:
        print("\n✅ Success! Processed Image:")
        result_image = Image.open(io.BytesIO(response.content))
        display(result_image)
        
        # Save result
        result_image.save("result.png")
        files.download("result.png")
        print("\n💾 Downloaded as result.png")
    else:
        print(f"\n❌ Error: {response.status_code}")
        print(response.text)
else:
    print("No file uploaded.")

---

## 📚 API Information

### Available Models

Send POST request to `/remove_background/` with:

**Parameters:**
- `file`: Image file (required)
- `method`: Model to use (required)
  - `tracer` - Tracer-B7 from Carvekit
  - `basnet` - BASNet from Carvekit
  - `rmbg` - RMBG-2.0 (Best for vehicles)

### Usage in Your Frontend

Update your local `.env` file:

```env
REACT_APP_TRACER_URL=<your_ngrok_url>
REACT_APP_BASNET_URL=<your_ngrok_url>
REACT_APP_RMBG_URL=<your_ngrok_url>
```

Then restart your React app:

```bash
npm start
```

### Example cURL Request

```bash
curl -X POST "<your_ngrok_url>/remove_background/" \
  -F "file=@image.jpg" \
  -F "method=rmbg" \
  --output result.png
```

### Important Notes

- ⚠️ **Session Limit**: Colab sessions expire after ~12 hours of inactivity
- ⚠️ **Ngrok Free Tier**: Limited to 40 connections/minute (upgrade for more)
- ⚠️ **Keep Running**: Don't stop the server cell while in use
- ✅ **New URL Each Time**: The ngrok URL changes when you restart the notebook

---

## 🎉 You're All Set!

Your background removal API is now running in the cloud for free!